In [ ]:
# 安装必要的库
# pip install ultralytics opencv-python matplotlib pandas numpy seaborn pillow ipython tqdm
#
# 如果需要使用GPU加速，还需安装相应版本的PyTorch
# pip install torch torchvision torchaudio
#
# 升级ultralytics和安装ray（用于并行处理）
# pip install --upgrade ultralytics ray

# ========== 使用说明 ========== #
"""
YOLOv8 目标检测训练和推理脚本

请直接修改脚本中的全局变量进行参数设置，包括数据集路径、输出路径、训练参数等。

主要参数说明:
- DATASET_ROOT: 数据集根目录
- DATA_YAML: 数据集YAML配置文件
- TRAIN_IMAGES_DIR: 训练图片目录
- TEST_IMAGES_DIR: 测试图片目录
- VALID_IMAGES_DIR: 验证图片目录
- OUTPUT_ROOT: 输出根目录
- EPOCHS: 训练轮数
- BATCH_SIZE: 批次大小
- OPTIMIZER: 优化器类型
- CONF_THRESHOLD: 置信度阈值
- IMAGE_SIZE: 图像大小
- PRETRAINED_MODEL: 预训练模型路径
"""

# ========== 库导入 ========== #
import os
import random
import pandas as pd
from PIL import Image
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

sns.set(style='darkgrid')
warnings.filterwarnings('ignore')

# 配置 Seaborn 图表的视觉风格
sns.set(rc={'axes.facecolor': '#eae8fa'}, style='darkgrid')

# ========== 路径和参数配置 ========== #
# 在这里修改数据集路径和输出路径以适应不同环境

# 数据集根目录/root/autodl-tmp/tt100k_yolo_clean
DATASET_ROOT = '/root/autodl-tmp/tt100k_yolo_clean/'  # 可根据实际情况修改

# 数据集相关路径
DATA_YAML = os.path.join(DATASET_ROOT, 'data.yaml')  # 数据集YAML配置文件
TRAIN_IMAGES_DIR = os.path.join(DATASET_ROOT, 'train/images')  # 训练图片目录
TEST_IMAGES_DIR = os.path.join(DATASET_ROOT, 'test/images')  # 测试图片目录
VALID_IMAGES_DIR = os.path.join(DATASET_ROOT, 'valid/images')  # 验证图片目录


# 输出根目录
OUTPUT_ROOT = '/root/pythonprojects/tt100k/ys/output'  # 可根据实际情况修改

# 模型输出路径
MODEL_OUTPUT_DIR = os.path.join(OUTPUT_ROOT, 'runs/detect/train')  # 模型训练输出目录
BEST_MODEL_PATH = os.path.join(MODEL_OUTPUT_DIR, 'weights/best.pt')  # 最佳模型权重路径

# 训练参数
EPOCHS = 200  # 训练轮数
BATCH_SIZE = 64  # 批次大小
OPTIMIZER = 'auto'  # 优化器类型
CONF_THRESHOLD = 0.5  # 置信度阈值
IMAGE_SIZE = 640  # 图像大小
PRETRAINED_MODEL = 'yolov8n.pt'  # 预训练模型路径




In [ ]:
import subprocess
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:


# ========== 功能函数 ========== #

# 辅助函数：获取目录中的第一张图片
def get_first_image(image_dir):
    """从目录中获取第一张JPG图片的路径"""
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not image_files:
        raise FileNotFoundError(f"在目录 {image_dir} 中没有找到图片文件")
    return os.path.join(image_dir, image_files[0])

# 3. 目标检测
def detect_objects(image_path, model_path="yolov8n.pt"):
    """使用 YOLOv8 进行目标检测"""
    model = YOLO(model_path)
    result = model.predict(source=image_path, imgsz=640)

    plot = result[0].plot()
    plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(plot))




def train_model(data_yaml, model_yaml, pretrained_weight, epochs=30, batch=8, optimizer='auto'):
    """
    训练 YOLOv8 模型，支持自定义模型结构 (yaml) 和基于官方预训练权重微调。

    Args:
        data_yaml (str): 数据集配置文件路径
        model_yaml (str): 模型结构文件路径 (比如 a.yaml 或 b.yaml)
        pretrained_weight (str): 预训练权重文件路径 (比如 yolov8n.pt)
        epochs (int): 训练轮数
        batch (int): 批大小
        optimizer (str): 优化器配置
    """

    # 1. 先加载官方预训练权重
    base_model = YOLO(pretrained_weight)

    # 2. 用自己的结构文件重新构建模型，同时从 base_model 中迁移已有权重
    model = YOLO(model_yaml)  # 注意这里是自己定义的 yaml

    # 3. 加载 base_model 的权重到 model（部分加载也OK）
    ckpt = base_model.ckpt  # ckpt 是 ultralytics 里权重内容
    model.model.load_state_dict(ckpt['model'].float().state_dict(), strict=False)
    print("[INFO] Successfully loaded pretrained weights from", pretrained_weight)

    # 4. 开始训练
    result = model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch,
        optimizer=optimizer
    )

    return model


# 5. 显示训练结果图像
def display_training_results(training_path, image_files):
    """显示训练过程中生成的结果图片"""
    for image_file in image_files:
        image_path = os.path.join(training_path, image_file)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(10, 10), dpi=120)
        plt.imshow(img)
        plt.axis('off')
        plt.show()


# 6. 画出训练结果曲线
def plot_training_curves(csv_path):
    """绘制训练损失和评估指标曲线"""
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()

    fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))
    sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0, 0])
    sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0, 1])
    sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1, 0])
    sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1, 1])
    sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2, 0])
    sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2, 1])
    sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3, 0])
    sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3, 1])
    sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4, 0])
    sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4, 1])

    plt.suptitle('训练指标和损失曲线', fontsize=24)
    plt.subplots_adjust(top=0.8)
    plt.tight_layout()
    plt.show()


# 7. 评估模型
def evaluate_model(model_path, split="val"):
    """评估训练好的模型"""
    model = YOLO(model_path)
    metrics = model.val(split=split)

    print("precision(B): ", metrics.results_dict["metrics/precision(B)"])
    print("recall(B): ", metrics.results_dict["metrics/recall(B)"])
    print("mAP50(B): ", metrics.results_dict["metrics/mAP50(B)"])
    print("mAP50-95(B): ", metrics.results_dict["metrics/mAP50-95(B)"])


# 8. 归一化图像
def normalize_image(image):
    """归一化图像"""
    return image / 255.0


# 9. 缩放图像
def resize_image(image, size=(640, 640)):
    """调整图像大小"""
    return cv2.resize(image, size)


# 10. 预测测试集图片
def predict_test_images(model_path, test_images_path):
    """在测试集上进行预测并显示结果"""
    model = YOLO(model_path)
    image_files = [file for file in os.listdir(test_images_path) if file.endswith('.jpg')]

    if len(image_files) > 0:
        num_images = len(image_files)
        step_size = max(1, num_images // 9)
        selected_images = [image_files[i] for i in range(0, num_images, step_size)]

        fig, axes = plt.subplots(3, 3, figsize=(20, 21))
        fig.suptitle('验证集预测结果', fontsize=24)

        for i, ax in enumerate(axes.flatten()):
            if i < len(selected_images):
                image_path = os.path.join(test_images_path, selected_images[i])
                image = cv2.imread(image_path)

                if image is not None:
                    resized_image = resize_image(image, size=(640, 640))
                    normalized_image = normalize_image(resized_image)
                    normalized_image_uint8 = (normalized_image * 255).astype(np.uint8)

                    results = model.predict(source=normalized_image_uint8, imgsz=640, conf=0.5)

                    annotated_image = results[0].plot(line_width=1)
                    annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
                    ax.imshow(annotated_image_rgb)
                else:
                    print(f"无法加载图片 {image_path}")
            ax.axis('off')

        plt.tight_layout()
        plt.show()


# 11. 导出模型
def export_model(model_path, export_format='onnx'):
    """导出模型为指定格式"""
    model = YOLO(model_path)
    model.export(format=export_format)


In [ ]:

# 默认使用预训练模型
model_path = PRETRAINED_MODEL

# 1. 目标检测
try:
    # 从训练图片目录中获取样本图片
    sample_image = get_first_image(TRAIN_IMAGES_DIR)
    detect_objects(sample_image, PRETRAINED_MODEL)
    print("步骤1(目标检测)完成")
except Exception as e:
    print(f"步骤1(目标检测)出错: {e}")


In [ ]:
# 2. 训练模型
try:
    print(f"开始训练模型，使用参数：epochs={EPOCHS}, batch_size={BATCH_SIZE}, optimizer={OPTIMIZER}")

    # 训练官方结构
    model = train_model(
    data_yaml=DATA_YAML,
    model_yaml="./ultralytics/cfg/models/v8/yolov8.yaml",
    pretrained_weight=PRETRAINED_MODEL,
    epochs=EPOCHS,
    batch=BATCH_SIZE
    )
    
        # 训练官方结构
    model = train_model(
    data_yaml=DATA_YAML,
    model_yaml="./ultralytics/cfg/models/v8/yolov8_cbam.yaml",
    pretrained_weight=PRETRAINED_MODEL,
    epochs=EPOCHS,
    batch=BATCH_SIZE
    )






    model_path = BEST_MODEL_PATH
    print(f"步骤2(训练模型)完成，使用训练后的模型：{model_path}")
except Exception as e:
    print(f"步骤2(训练模型)出错: {e}")

In [ ]:

# 3. 显示训练结果
try:
    if os.path.exists(MODEL_OUTPUT_DIR):
        display_training_results(MODEL_OUTPUT_DIR, [
            'confusion_matrix_normalized.png',
            'results.png'
        ])
        print("步骤3(显示训练结果)完成")
    else:
        print("步骤3(显示训练结果)被跳过 - 结果目录不存在")
except Exception as e:
    print(f"步骤3(显示训练结果)出错: {e}")

# 4. 显示训练结果曲线
try:
    results_csv = os.path.join(MODEL_OUTPUT_DIR, 'results.csv')
    if os.path.exists(results_csv):
        plot_training_curves(results_csv)
        print("步骤4(显示训练结果曲线)完成")
    else:
        print("步骤4(显示训练结果曲线)被跳过 - 结果文件不存在")
except Exception as e:
    print(f"步骤4(显示训练结果曲线)出错: {e}")

In [ ]:

# 5. 评估模型
try:
    evaluate_model(model_path)
    print(f"步骤5(评估模型)完成，使用模型: {model_path}")
except Exception as e:
    print(f"步骤5(评估模型)出错: {e}")

# 6. 预测测试集图片
try:
    predict_test_images(model_path, TEST_IMAGES_DIR)
    print(f"步骤6(预测测试集图片)完成，使用模型: {model_path}")
except Exception as e:
    print(f"步骤6(预测测试集图片)出错: {e}")

# 7. 导出模型
try:
    export_model(model_path)
    print(f"步骤7(导出模型)完成，导出模型: {model_path}")
except Exception as e:
    print(f"步骤7(导出模型)出错: {e}")

print("所有流程执行完毕！")